# Vault Client Count based on Ranjit

This tutorial is based on great work done by Ranjit. You can find his original repo here. [GitHub - WhatsARanjit/vault-counter](https://github.com/WhatsARanjit/vault-counter)

## Overview

Simple container script to run and count HashiCorp Vault entities, roles/users, and tokens created without entities.

* The script will drill into child namespaces below the supplied namespace input.

This tutorial also helps you start up a Vault server and fill it synthetically with tokens and entities.

If you have an environment already set up, then just do the prerequisites and skip to the [Get Count](#Get-Count---Using-container) section 

## Prerequisites

* Software and versions
* Credentials
* Hardware

### Set Main Environment Variables

In [ ]:
export RED="\e[0;31m" YELLOW="\e[0;33m" BLDYELLOW="\e[1;33m" GREEN="\e[0;32m"
export CYAN="\e[0;36m" BLUE="\e[0;34m" WHITE="\e[0;37m" BLDWHITE="\e[1;37m"
export NC="\e[0m"
# Some commands may have sensitive information. Prevent commands starting with a space to be saved to shell history.
HISTCONTROL=ignoreboth # do not save lines that begin with space in history

printf "${GREEN}# Set MAIN_DIR for future reference.${NC}\n"
export MAIN_DIR=${PWD}

Be sure to set the following variables with your own values, especially `VAULT_ADDR` and `VAULT_TOKEN`.

In [ ]:
printf "${GREEN}# Set Vault Variables.${NC}\n"
export VAULT_PORT=8205
# export VAULT_TOKEN=s.lq3fLt00S9SmVB3S0iOQ1Jpb
export VAULT_TOKEN=root
export VAULT_ADDR=http://192.168.17.170:${VAULT_PORT:-8200}
# export VAULT_LICENSE=$(cat ../../license/vault.hclic)
echo $VAULT_ADDR
# Set dir env vars. Create required directory.
WORK_DIR=config/vault

In [ ]:
mkdir -p config/vault/{data,logs,config}

## Steps - Demo Setup

### Start Vault Server in Dev Mode

Start the vault process in the background.

In [ ]:
docker run -d --rm --name vault \
    --cap-add IPC_LOCK \
    -p 8205:8200 \
    -e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
    -e 'VAULT_DEV_LISTEN_ADDRESS=0.0.0.0:8200' \
    hashicorp/vault-enterprise:1.7.5_ent
    # -e "VAULT_ADDR=http://127.0.0.1:8200" \

Configure environment variables for connecting to Vault. We've set the dev mode root token to "`${VAULT_TOKEN}`".

In [ ]:
docker logs vault

Check vault status

In [ ]:
vault status

### Create non-entity tokens

Create some tokens

In [ ]:
for i in {1..30}; do
    vault token create -policy=my-policy -policy=other-policy > /dev/null 2>&1
done

### Userpass

In [ ]:
vault auth enable userpass

Create Users 10-19 but **don't** login. This will create Users but not Entities within Vault. 

In [ ]:
for i in {10..19}; do
    vault write auth/userpass/users/user${i} password="password${i}" policies="default"
    # vault login -method=userpass username=user${i} password=password${i}
done

Create Users 10-19 and login. This will create Users and Entities within Vault .

In [ ]:
for i in {10..19}; do
    vault write auth/userpass/users/user${i} password="password${i}" policies="default"
    vault login -method=userpass username=user${i} password=password${i} > /dev/null 2>&1
done

### AppRole

Create AppRole and login

In [ ]:
vault auth enable approle

Create AppRole but **don't** login. This will create AppRoles but not Entities within Vault.

In [ ]:
for i in {10..19}; do
    vault write -f auth/approle/role/app${i}
    # vault write auth/approle/login \
    # role_id=$(vault read -field=role_id auth/approle/role/app${i}/role-id) \
    # secret_id=$(vault write -force -field=secret_id auth/approle/role/app${i}/secret-id)
done

Create AppRole and login. This will create AppRoles and Entities within Vault.

In [ ]:
for i in {20..29}; do
    vault write -f auth/approle/role/app${i}
    vault write auth/approle/login \
    role_id=$(vault read -field=role_id auth/approle/role/app${i}/role-id) \
    secret_id=$(vault write -force -field=secret_id auth/approle/role/app${i}/secret-id) \
    > /dev/null 2>&1
done

## Get Count - Using container

In [ ]:
docker run --rm --name vault_counter \
    -e VAULT_ADDR=${VAULT_ADDR} \
    -e VAULT_TOKEN=${VAULT_TOKEN} \
    -e JSON_OUTPUT="" \
    -e SKIP_ORPHAN_TOKENS="" \
    -e CURL_VERBOSE="" \
    -e VAULT_NAMESPACE=root \
    -e VAULT_CLIENT_CERT="" \
    -e VAULT_CLIENT_KEY="" \
    -e VAULT_CACERT="" \
    whatsaranjit/vault_counter:latest

| Variable | Description | Default |
| --- | --- | --- |
| `CURL_VERBOSE=""` | Set to anything to add the `-v` flag to cURL statements. | Default: `null` |
| `JSON_OUTPUT=""` | Set to anything for json output |  Default: `null`
| `VAULT_ADDR=${VAULT_ADDR}` | The URL to your Vault server <br> - I'm using my host's IP for the `counter` container to reach the `vault` container. <br> - Do not use `localhost` or `127.0.0.1`. This will cause the container to query itself and not your machine. | Default: http://127.0.0.1:8200
| `VAULT_CACERT` <br> `VAULT_CLIENT_CERT` <br> `VAULT_CLIENT_KEY` | | Default: `null`
| `VAULT_NAMESPACE=root` | Namespace within Vault to count along with all child namespaces. | Default: `root`
| `SKIP_ORPHAN_TOKENS=""` | Set to anything to skip inspecting each token to see if it is tied to an auth method. <br> **RECOMMENDED** if you have lots of tokens.

**TIPS**
* Start with `SKIP_ORPHAN_TOKENS` enabled to get a feel for impact on system.
* This script can take down a node that has lots of tokens and not enough HW resources.

Sample Output
```shell
Vault address: http://192.168.17.167:8200
Namespace: root/
Total entities: 0
Total users/roles: 30
Total tokens: 91
Total orphan tokens: 91
```

## Get Count - Using script

Save script to `/tmp` directory.

In [ ]:
curl -o /tmp/counter.sh \
  https://raw.githubusercontent.com/WhatsARanjit/vault-counter/master/scripts/counter.sh \
  && chmod +x /tmp/counter.sh

In [ ]:
export JSON_OUTPUT=""
export SKIP_ORPHAN_TOKENS=""
export VAULT_CLIENT_CERT=""
export VAULT_CLIENT_KEY=""
export VAULT_CACERT=""
export VAULT_NAMESPACE=root

In [ ]:
/tmp/counter.sh

## Conclusion

When you want to purchase that cool Vault Enterprise solution. One of the key factors for licensing is client count. Luckily for Vault 1.7 and higher, there are metrics built-in that can provide you the client count. For previous versions, there is a tool that can scan your audit logs, if you turned on audit logging. For everyone else, the `vault_counter` tool can provide you a client count.

## Clean up

### Delete artifacts

In [ ]:
rm /tmp/counter.sh

### Stop Vault container

In [ ]:
docker stop vault
docker rm vault # should fail since we used --rm flag

### Remove Docker Image

In [ ]:
docker image rm whatsaranjit/vault_counter

In [ ]:
docker image rm vault